In [1]:
from config import retinanet_config as config
from bs4 import BeautifulSoup
from imutils import paths
import argparse
import random
import os

In [2]:
import easydict
args = easydict.EasyDict({
        "annotations": config.ANNOT_PATH,
        "images": config.IMAGES_PATH,
        "train": config.TRAIN_CSV,
        "test": config.TEST_CSV,
        "classes": config.CLASSES_CSV,
        "split": config.TRAIN_TEST_SPLIT
})



In [3]:

annot_path = args["annotations"]
images_path = args["images"]
train_csv = args["train"]
test_csv = args["test"]
classes_csv = args["classes"]
train_test_split = args["split"]

In [4]:

imagePaths = list(paths.list_files(images_path))
random.shuffle(imagePaths)
i = int(len(imagePaths) * train_test_split)
trainImagePaths = imagePaths[:i]
testImagePaths = imagePaths[i:]

In [8]:
print(i)
print(int(len(imagePaths)))

63
85


In [6]:

dataset = [ ("train", trainImagePaths, train_csv),
("test", testImagePaths, test_csv)]

In [7]:
# initialize the set of classes we have
CLASSES = set()

In [9]:
# loop over the datasets
for (dType, imagePaths, outputCSV) in dataset:
 
    print ("[INFO] creating '{}' set...".format(dType))
    print ("[INFO] {} total images in '{}' set".format(len(imagePaths), dType))

   
    csv = open(outputCSV, "w")

    # loop over the image paths
    for imagePath in imagePaths:
        # build the corresponding annotation path
        fname = imagePath.split(os.path.sep)[-1]
        fname = "{}.xml".format(fname[:fname.rfind(".")])
        annotPath = os.path.sep.join([annot_path, fname])

        # load the contents of the annotation file and buid the soup
        contents = open(annotPath).read()
        soup = BeautifulSoup(contents, "html.parser")

        # extract the image dimensions
        w = int(soup.find("width").string)
        h = int(soup.find("height").string)

        # loop over all object elements
        for o in soup.find_all("object"):
            #extract the label and bounding box coordinates
            label = o.find("name").string
            xMin = int(float(o.find("xmin").string))
            yMin = int(float(o.find("ymin").string))
            xMax = int(float(o.find("xmax").string))
            yMax = int(float(o.find("ymax").string))

          
            xMin = max(0, xMin)
            yMin = max(0, yMin)
            xMax = min(w, xMax)
            yMax = min(h, yMax)

           
            if xMin >= xMax or yMin >= yMax:
                continue
            elif xMax <= xMin or yMax <= yMin:
                continue

            # write the image path, bb coordinates, label to the output CSV
            row = [os.path.abspath(imagePath),str(xMin), str(yMin), str(xMax),
                    str(yMax), str(label)]
            csv.write("{}\n".format(",".join(row)))

            # update the set of unique class labels
            CLASSES.add(label)

  
    csv.close()

# write the classes to file
print("[INFO] writing classes...")
csv = open(classes_csv, "w")
rows = [",".join([c, str(i)]) for (i,c) in enumerate(CLASSES)]
csv.write("\n".join(rows))
csv.close()

[INFO] creating 'train' set...
[INFO] 63 total images in 'train' set
[INFO] creating 'test' set...
[INFO] 22 total images in 'test' set
[INFO] writing classes...


In [10]:
print(CLASSES)

{'car'}


In [ ]:
#Below code section was written to train the model in a different way but this option also did not work. Can be used in future for improvements.

In [1]:
#import numpy as np
#import os
#import matplotlib.pyplot as plt

#!pip install opencv-python
#import cv2

#import keras

#import tensorflow as tf
#from tensorflow.keras.datasets import cifar10
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

#from keras_retinanet import models
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
#from tensorflow.keras.callbacks import TensorBoard
#import pickle
#import time
#import random

Using TensorFlow backend.


In [2]:
#def get_session():
#    config = tf.ConfigProto()
#    config.gpu_options.allow_growth = True
#    return tf.Session(config=config)

In [3]:
#keras.backend.tensorflow_backend.set_session(get_session())


In [3]:
#num_classes = 15


#from keras_retinanet.models.resnet import ResNetBackbone
#model = ResNetBackbone('resnet50').retinanet(num_classes)
#model.load_weights('/home/user1/Desktop/sdc_project/megha_retinanet/resnet50_coco_best_v2.1.0.h5', by_name=True, skip_mismatch=True)

#model.compile(
 #       loss={
  #          'regression'    : '/home/user1/Desktop/sdc_project/megha_retinanet/keras-retinanet/keras_retinanet.losses.smooth_l1()',
   #         'classification': '/home/user1/Desktop/sdc_project/megha_retinanet/keras-retinanet/keras_retinanet.losses.focal()'
    #        
     #   },
      #  optimizer=keras.optimizers.adam(lr=1e-4, clipnorm=0.001),
    #) 

#model.fit('/home/user1/Desktop/sdc_project/megha_retinanet/dataset/train.csv','/home/user1/Desktop/sdc_project/megha_retinanet/dataset/classes.csv', batch_size=3, epochs=5, validation_split=0.1, callbacks=['/home/user1/Desktop/sdc_project/megha_retinanet/tensorboard'])

In [7]:
#model_path = os.path.join('..', '/home/user1/Desktop/sdc_project/megha_retinanet/', 'resnet50_coco_best_v2.1.0.h5')
#model = models.load_model(model_path, backbone_name='resnet50')


In [8]:
#labels_to_names ={0:'car'}

In [2]:
#print(model.summary())
#print(labels_to_names)

In [23]:
#Below code is to draw the boxes on the vehicles detected

#from keras_retinanet import models
#from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
#from keras_retinanet.utils.visualization import draw_box, draw_caption
#from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
#import matplotlib.pyplot as plt
#import cv2
#import os
#import numpy as np
#import time


In [1]:
# load image
#image = read_image_bgr('dataset/images/video_51.png')

# copy to draw on
#draw = image.copy()
#draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
#image = preprocess_image(image)
#image, scale = resize_image(image)

# process image
#start = time.time()
#boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
#print("processing time: ", time.time() - start)

# correct for image scale
#boxes /= scale

# visualize detections
#for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
 #   if score < 0.5:
  #      print('less than 0.5')
   #     break
        
    #color = label_color(label)
    #print(score)
    #print(label)
    #print(box)
    #b = box.astype(int)
    #draw_box(draw, b, color=color)
    #print(labels_to_names[0])
    #caption = "{} {:.3f}".format(labels_to_names[0], score)
    #draw_caption(draw, b, caption)
    
#plt.figure(figsize=(15, 15))
#plt.axis('off')
#plt.imshow(draw)
#plt.show()